In [30]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [ ]:
path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-KERNEL-Int-'

In [ ]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "06-00-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [31]:
%%log_time 1

nameFileV = []
pathFileV = []
rootFileV = []
nameFileM = []
pathFileM = []
rootFileM = []
nameFileS = []
pathFileS = []
rootFileS = []
nameFileSV = []
pathFileSV = []
rootFileSV = []

path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT'

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("ScalarSize.csv"):
            nameFileS.append(file)
            pathFileS.append(os.path.join(root, file))
            rootFileS.append(root)
        elif file.endswith("ScalarVariable.csv"):
            nameFileSV.append(file)
            pathFileSV.append(os.path.join(root, file))
            rootFileSV.append(root)
        elif file.endswith("VectorNew.csv"):
            nameFileV.append(file)
            pathFileV.append(os.path.join(root, file))
            rootFileV.append(root)
        elif file.endswith("MatrixNew.csv"):
            nameFileM.append(file)
            pathFileM.append(os.path.join(root, file))
            rootFileM.append(root)

In [39]:
len(pathFileS)

178

In [40]:
len(pathFileSV)

60

In [41]:
len(pathFileV)

80

In [42]:
len(pathFileM)

118

In [29]:
'''
for i in range(0, len(pathFileS)):
    if os.path.exists(pathFileS[i]):
        os.remove(pathFileS[i])
        
for i in range(0, len(pathFileSV)):
    if os.path.exists(pathFileSV[i]):
        os.remove(pathFileSV[i])
        
for i in range(0, len(pathFileV)):
    if os.path.exists(pathFileV[i]):
        os.remove(pathFileV[i])
        
for i in range(0, len(pathFileM)):
    if os.path.exists(pathFileM[i]):
        os.remove(pathFileM[i])
'''

In [43]:
%%log_time 2

# ScalarSize file
for i in range(0, len(pathFileS)):
    path = pathFileS[i]
    try:
        dfSTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)

    # dfSTemp = dfSTemp.rename(columns={dfSTemp.columns[0]:'ID_VAL'})    
    
    for j in range(1, len(dfSTemp.columns)):
        dfSTemp = dfSTemp.rename(columns={dfSTemp.columns[j]:'SS_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileS[i].split('\\')
    else:
        spR = rootFileS[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfSTemp['DEVICE'] = device
    dfSTemp['BOARD'] = board
    dfSTemp['FUNCTION'] = function
    dfSTemp['DATA_TYPE'] = datatype
    # dfSTemp['OPT_FLAG'] = optFlag
    # dfSTemp['MV_SIZE'] = matrixVectorSize
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(str)
    dfSTemp['inputName'] = dfSTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(int)
    dfSTemp.sort_values('inputName')
    
    if i == 0:
        dfSTot = dfSTemp
    else:
        frames = [dfSTot, dfSTemp]
        dfSTot = pd.concat(frames)
        
dfSTot = dfSTot.fillna(0)

In [44]:
dfSTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,11,8,Leon3,Leon3,binary_search,int16_t
1,19,8,Leon3,Leon3,binary_search,int16_t
2,12,8,Leon3,Leon3,binary_search,int16_t
3,38,23,Leon3,Leon3,binary_search,int16_t
4,81,6,Leon3,Leon3,binary_search,int16_t
...,...,...,...,...,...,...
95,3,3,RiscV,RiscV,astar,int8_t
96,98,11,RiscV,RiscV,astar,int8_t
97,29,26,RiscV,RiscV,astar,int8_t
98,80,27,RiscV,RiscV,astar,int8_t


In [45]:
%%log_time 3

# ScalarVariable file
for i in range(0, len(pathFileSV)):
    path = pathFileSV[i]
    try:
        dfSVTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)
    
    # dfSVTemp = dfSVTemp.rename(columns={dfSVTemp.columns[0]:'ID_VAL'})
    
    for j in range(1, len(dfSVTemp.columns)):
        dfSVTemp = dfSVTemp.rename(columns={dfSVTemp.columns[j]:'SV_VAL_'+str(j)})
        
    if os.name == 'nt':
        spR = rootFileSV[i].split('\\')
    else:
        spR = rootFileSV[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfSVTemp['DEVICE'] = device
    dfSVTemp['BOARD'] = board
    dfSVTemp['FUNCTION'] = function
    dfSVTemp['DATA_TYPE'] = datatype
    # dfSVTemp['OPT_FLAG'] = optFlag
    # dfSVTemp['MV_SIZE'] = matrixVectorSize
    dfSVTemp['inputName'] = dfSVTemp['inputName'].astype(str)
    dfSVTemp['inputName'] = dfSVTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfSVTemp['inputName'] = dfSVTemp['inputName'].astype(int)
    dfSVTemp.sort_values('inputName')
    
    if i == 0:
        dfSVTot = dfSVTemp
    else:
        frames = [dfSVTot, dfSVTemp]
        dfSVTot = pd.concat(frames)

dfSVTot = dfSVTot.fillna(0)

In [47]:
dfSVTot

,inputName,SV_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE,SV_VAL_2
0,11,17299,Leon3,Leon3,binary_search,int16_t,0.0
1,19,18887,Leon3,Leon3,binary_search,int16_t,0.0
2,12,-5112,Leon3,Leon3,binary_search,int16_t,0.0
3,38,-3980,Leon3,Leon3,binary_search,int16_t,0.0
4,81,17299,Leon3,Leon3,binary_search,int16_t,0.0
...,...,...,...,...,...,...,...
95,3,67,RiscV,RiscV,gcd,int8_t,107.0
96,98,56,RiscV,RiscV,gcd,int8_t,127.0
97,29,11,RiscV,RiscV,gcd,int8_t,21.0
98,80,86,RiscV,RiscV,gcd,int8_t,112.0


In [48]:
%%log_time 4

# Vector file
for i in range(0, len(pathFileV)):
    path = pathFileV[i]
    dfVTemp = pd.read_csv(path, sep=',',engine='python')
    
    dfVTemp = dfVTemp.rename(columns={dfVTemp.columns[0]:'inputName'})
    
    for j in range(1, len(dfVTemp.columns)):
        dfVTemp = dfVTemp.rename(columns={dfVTemp.columns[j]:'V_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileV[i].split('\\')
    else:
        spR = rootFileV[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfVTemp['DEVICE'] = device
    dfVTemp['BOARD'] = board
    dfVTemp['FUNCTION'] = function
    dfVTemp['DATA_TYPE'] = datatype
    # dfVTemp['OPT_FLAG'] = optFlag
    # dfVTemp['MV_SIZE'] = matrixVectorSize
    # dfVTemp['ID_VAL'] = dfVTemp['ID_VAL'].astype(str)
    dfVTemp['inputName'] = dfVTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfVTemp['inputName'] = dfVTemp['inputName'].astype(int)
    dfVTemp.sort_values('inputName')
    
    if i == 0:
        dfVTot = dfVTemp
    else:
        frames = [dfVTot, dfVTemp]
        dfVTot = pd.concat(frames)

dfVTot = dfVTot.fillna(0)

In [49]:
dfVTot

,inputName,V_VAL_1,V_VAL_2,V_VAL_3,V_VAL_4,V_VAL_5,V_VAL_6,V_VAL_7,V_VAL_8,V_VAL_9,...,V_VAL_27,V_VAL_28,V_VAL_29,V_VAL_30,V_VAL_31,V_VAL_32,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,19,-9904,-27021,7732,-25558,-19977,-26764,-2569,-1983,0,...,0,0,0,0,0,0,Leon3,Leon3,binary_search,int16_t
1,12,21320,14476,-10633,21872,30121,2823,-26699,23587,0,...,0,0,0,0,0,0,Leon3,Leon3,binary_search,int16_t
2,38,-14617,-31102,4784,15515,10270,5132,-29872,29029,-929,...,0,0,0,0,0,0,Leon3,Leon3,binary_search,int16_t
3,81,20544,25719,-11393,31379,7089,6847,0,0,0,...,0,0,0,0,0,0,Leon3,Leon3,binary_search,int16_t
4,5,8028,20065,3617,4578,8869,-29395,19949,-28898,16619,...,0,0,0,0,0,0,Leon3,Leon3,binary_search,int16_t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3,46,-20,72,-80,-102,85,-29,112,-107,...,0,0,0,0,0,0,RiscV,RiscV,mergesort,int8_t
95,98,39,-89,97,0,0,0,0,0,0,...,0,0,0,0,0,0,RiscV,RiscV,mergesort,int8_t
96,29,46,83,127,-43,-33,-13,17,-123,59,...,0,0,0,0,0,0,RiscV,RiscV,mergesort,int8_t
97,80,-54,-62,-128,15,-106,71,71,-98,83,...,0,0,0,0,0,0,RiscV,RiscV,mergesort,int8_t


In [50]:
%%log_time 5

# Matrix file
pathFileM.pop(0)
rootFileM.pop(0)

for i in range(0, len(pathFileM)):
    path = pathFileM[i]
    dfMTemp = pd.read_csv(path, sep=',',engine='python')
    
    dfMTemp = dfMTemp.rename(columns={dfMTemp.columns[0]:'inputName'})
    
    for j in range(1, len(dfMTemp.columns)):
        dfMTemp = dfMTemp.rename(columns={dfMTemp.columns[j]:'M_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileM[i].split('\\')
    else:
        spR = rootFileM[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfMTemp['DEVICE'] = device
    dfMTemp['BOARD'] = board
    dfMTemp['FUNCTION'] = function
    dfMTemp['DATA_TYPE'] = datatype
    # dfMTemp['OPT_FLAG'] = optFlag
    # dfMTemp['MV_SIZE'] = matrixVectorSize
    # dfMTemp['ID_VAL'] = dfVTemp['ID_VAL'].astype(str)
    dfMTemp['inputName'] = dfMTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfMTemp['inputName'] = dfMTemp['inputName'].astype(int)
    dfMTemp.sort_values('inputName')
    
    if i == 0:
        dfMTot = dfMTemp
    else:
        frames = [dfMTot, dfMTemp]
        dfMTot = pd.concat(frames)

dfMTot = dfMTot.fillna(0)

In [51]:
dfMTot

,inputName,M_VAL_1,M_VAL_2,M_VAL_3,M_VAL_4,M_VAL_5,M_VAL_6,M_VAL_7,M_VAL_8,M_VAL_9,...,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,19,0.000000,2.172621e+07,2.294292e+08,8.246974e+07,2.702828e+08,1.989554e+08,1.586817e+08,1.195475e+08,0.000000,...,3.412183e+06,0.000000e+00,4.604590e+06,8.698447e+05,5.247350e+06,2.421325e+06,Leon3,Leon3,floydwarshall,int32_t
1,12,0.000000,2.428242e+07,8.655686e+08,6.661599e+08,1.007845e+09,6.727025e+08,4.495055e+08,1.527222e+08,0.000000,...,1.174354e+08,1.017165e+08,3.323411e+08,8.389071e+07,1.987576e+08,1.411547e+08,Leon3,Leon3,floydwarshall,int32_t
2,38,0.000000,1.068461e+08,1.211895e+09,1.770290e+08,1.051842e+09,9.400396e+08,9.895666e+08,2.733612e+08,0.000000,...,4.147273e+08,2.693391e+07,4.756589e+08,6.942416e+08,5.224764e+08,2.222252e+08,Leon3,Leon3,floydwarshall,int32_t
3,81,0.000000,3.004586e+08,9.802056e+08,9.815204e+07,6.966458e+08,1.064586e+09,9.138124e+08,5.038792e+08,0.000000,...,3.513024e+08,4.422283e+08,6.068235e+08,3.214186e+08,4.600182e+08,9.391816e+07,Leon3,Leon3,floydwarshall,int32_t
4,5,0.000000,3.586937e+08,1.421636e+09,5.116215e+08,9.377901e+08,1.009724e+09,1.004611e+09,4.254581e+08,0.000000,...,5.384789e+08,2.462995e+08,8.919498e+08,5.711683e+08,7.899630e+08,1.096830e+08,Leon3,Leon3,floydwarshall,int32_t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3,0.000000,9.816701e-03,1.646509e+00,2.145813e+00,4.724091e+00,5.643633e-01,1.694859e+00,2.898039e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,RiscV,RiscV,astar,int8_t
95,98,0.000000,9.634695e-01,1.676160e+01,1.457666e+01,2.117285e+01,1.761368e+01,2.221247e+01,5.017455e+00,0.000000,...,2.798929e+00,1.052588e+00,2.840234e+00,2.946223e+00,1.506112e+00,1.447239e+00,RiscV,RiscV,astar,int8_t
96,29,0.000000,1.221557e+01,5.961680e+01,1.572566e+01,6.055743e+01,4.457347e+01,5.872071e+01,2.106198e+01,12.880631,...,3.920923e+01,2.256775e+01,3.344414e+01,3.321075e+01,7.242013e+01,2.016718e+01,RiscV,RiscV,astar,int8_t
97,80,3.780873,1.889049e+01,5.526664e+01,1.594926e+01,6.377784e+01,3.965177e+01,7.061414e+01,1.954043e+01,0.000000,...,4.256205e+01,2.595560e+01,5.093166e+01,3.308801e+01,4.218966e+01,2.247050e+01,RiscV,RiscV,astar,int8_t


In [52]:
dfSTot = dfSTot.reset_index(drop=True)
dfSVTot = dfSVTot.reset_index(drop=True)
dfVTot = dfVTot.reset_index(drop=True)
dfMTot = dfMTot.reset_index(drop=True)

In [53]:
dfSTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,11,8,Leon3,Leon3,binary_search,int16_t
1,19,8,Leon3,Leon3,binary_search,int16_t
2,12,8,Leon3,Leon3,binary_search,int16_t
3,38,23,Leon3,Leon3,binary_search,int16_t
4,81,6,Leon3,Leon3,binary_search,int16_t
...,...,...,...,...,...,...
17795,3,3,RiscV,RiscV,astar,int8_t
17796,98,11,RiscV,RiscV,astar,int8_t
17797,29,26,RiscV,RiscV,astar,int8_t
17798,80,27,RiscV,RiscV,astar,int8_t


In [54]:
%%log_time 6

# To get the common columns between the DataFrames, we can use set intersection
common_columns = set(dfSTot.columns).intersection(set(dfSVTot.columns)).intersection(set(dfVTot.columns)).intersection(set(dfMTot.columns))

# Convert the intersection to a list
common_columns = list(common_columns)

# Merge the DataFrames using the common columns
dfTot = pd.merge(dfSTot, dfSVTot, on=common_columns, how='left', sort=False)
dfTot = pd.merge(dfTot, dfVTot, on=common_columns, how='left', sort=False) 
dfTot = pd.merge(dfTot, dfMTot, on=common_columns, how='left', sort=False) 

# Display the resulting DataFrame
# print(dfTot)

In [ ]:
# dfSTot.to_csv("dfSTot.csv", sep=';')
# dfSVTot.to_csv("dfSVTot.csv", sep=';')
# dfVTot.to_csv("dfVTot.csv", sep=';')
# dfMTot.to_csv("dfMTot.csv", sep=';')

In [55]:
dfTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE,SV_VAL_1,SV_VAL_2,V_VAL_1,V_VAL_2,...,M_VAL_63,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72
0,11,8,Leon3,Leon3,binary_search,int16_t,17299.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,8,Leon3,Leon3,binary_search,int16_t,18887.0,0.0,-9904.0,-27021.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,8,Leon3,Leon3,binary_search,int16_t,-5112.0,0.0,21320.0,14476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38,23,Leon3,Leon3,binary_search,int16_t,-3980.0,0.0,-14617.0,-31102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,81,6,Leon3,Leon3,binary_search,int16_t,17299.0,0.0,20544.0,25719.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17795,3,3,RiscV,RiscV,astar,int8_t,14.0,0.0,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17796,98,11,RiscV,RiscV,astar,int8_t,158.0,0.0,NaN,NaN,...,4.418751,4.112515,0.000000,0.230080,2.798929,1.052588,2.840234,2.946223,1.506112,1.447239
17797,29,26,RiscV,RiscV,astar,int8_t,11.0,0.0,NaN,NaN,...,58.320614,23.503172,0.000000,24.357084,39.209229,22.567751,33.444138,33.210751,72.420128,20.167185
17798,80,27,RiscV,RiscV,astar,int8_t,255.0,0.0,NaN,NaN,...,52.871029,13.383142,8.783592,0.000000,42.562054,25.955601,50.931664,33.088013,42.189663,22.470501


In [56]:
%%log_time 7

dfTot = dfTot[ ['inputName','DEVICE','BOARD','FUNCTION','DATA_TYPE'] + [ col for col in dfTot.columns if (col != 'inputName' and col != 'DEVICE' and col != 'BOARD' and col != 'FUNCTION' and col != 'DATA_TYPE') ] ]

#### Realingn column

In [57]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,SS_VAL_1,SV_VAL_1,SV_VAL_2,V_VAL_1,V_VAL_2,...,M_VAL_63,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72
0,11,Leon3,Leon3,binary_search,int16_t,8,17299.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,Leon3,Leon3,binary_search,int16_t,8,18887.0,0.0,-9904.0,-27021.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,Leon3,Leon3,binary_search,int16_t,8,-5112.0,0.0,21320.0,14476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38,Leon3,Leon3,binary_search,int16_t,23,-3980.0,0.0,-14617.0,-31102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,81,Leon3,Leon3,binary_search,int16_t,6,17299.0,0.0,20544.0,25719.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17795,3,RiscV,RiscV,astar,int8_t,3,14.0,0.0,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17796,98,RiscV,RiscV,astar,int8_t,11,158.0,0.0,NaN,NaN,...,4.418751,4.112515,0.000000,0.230080,2.798929,1.052588,2.840234,2.946223,1.506112,1.447239
17797,29,RiscV,RiscV,astar,int8_t,26,11.0,0.0,NaN,NaN,...,58.320614,23.503172,0.000000,24.357084,39.209229,22.567751,33.444138,33.210751,72.420128,20.167185
17798,80,RiscV,RiscV,astar,int8_t,27,255.0,0.0,NaN,NaN,...,52.871029,13.383142,8.783592,0.000000,42.562054,25.955601,50.931664,33.088013,42.189663,22.470501


In [58]:
%%log_time 8

dfTot.to_csv("TotalInput.csv", sep=';', index=False, encoding='utf-8')

In [21]:
####################### OLD CODE #############################

In [102]:
import pandas as pd
import numpy as np

'''
# Add Zeros
def merge_dataframes_fillna(df1, df2, merge_on):
    # Perform the merge with suffixes to handle duplicate columns
    merged_df = pd.merge(df1, df2, how='outer', on=merge_on, suffixes=('_df1', '_df2'))

    # Identify columns with suffixes
    suffixes = ('_df1', '_df2')
    common_columns = [col.replace(suffixes[0], '') for col in merged_df.columns if col.endswith(suffixes[0])]

    # Combine duplicate columns by filling missing values with zero
    for col in common_columns:
        col_df1 = f"{col}{suffixes[0]}"
        col_df2 = f"{col}{suffixes[1]}"
        
        # Convert columns to numeric type, if possible
        merged_df[col_df1] = pd.to_numeric(merged_df[col_df1], errors='coerce').fillna(0)
        merged_df[col_df2] = pd.to_numeric(merged_df[col_df2], errors='coerce').fillna(0)
        
        # Sum the values of the converted columns
        merged_df[col] = merged_df[col_df1] + merged_df[col_df2]
    
    # Remove the columns with suffixes
    columns_to_drop = [f"{col}{suffixes[0]}" for col in common_columns] + [f"{col}{suffixes[1]}" for col in common_columns]
    merged_df.drop(columns=columns_to_drop, inplace=True)

    # Fill missing values in other columns with zero
    merged_df.fillna(0, inplace=True)
    
    return merged_df
'''

def make_columns_unique(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

# Add NaN
def merge_dataframes_fillna(df1, df2, merge_on):
    # Make sure column names are unique in both DataFrames
    df1 = make_columns_unique(df1)
    df2 = make_columns_unique(df2)
    
    # Perform the merge with suffixes to handle duplicate columns
    merged_df = pd.merge(df1, df2, how='outer', on=merge_on, suffixes=('_df1', '_df2'))

    # Identify columns with suffixes
    suffixes = ('_df1', '_df2')
    common_columns = [col.replace(suffixes[0], '') for col in merged_df.columns if col.endswith(suffixes[0])]

    # Combine duplicate columns by filling missing values with NaN
    for col in common_columns:
        col_df1 = f"{col}{suffixes[0]}"
        col_df2 = f"{col}{suffixes[1]}"
        
        # Combine the columns, filling NaN where data is missing
        merged_df[col] = merged_df[col_df1].combine_first(merged_df[col_df2])

    # Remove the columns with suffixes
    columns_to_drop = [f"{col}{suffixes[0]}" for col in common_columns] + [f"{col}{suffixes[1]}" for col in common_columns]
    merged_df.drop(columns=columns_to_drop, inplace=True)

    return merged_df

In [103]:
merge_on = ['ID_VAL','DEVICE','BOARD','FUNCTION','DATA_TYPE','OPT_FLAG','MV_SIZE']

In [104]:
dfTot1 = merge_dataframes_fillna(dfSTot, dfSVTot, merge_on)

In [105]:
dfTot1

,ID_VAL,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,SS_VAL_1,SV_VAL_1
0,11,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,17.0,NaN
1,12,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,10.0,NaN
2,19,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,18.0,NaN
3,38,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,2.0,NaN
4,73,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,7.0
5,73,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,7.0
6,73,Arm,Arm,astar,int8_t,optO1-02,32x32,NaN,7.0
7,80,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,214.0
8,80,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,214.0
9,81,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,22.0,NaN


In [106]:
dfTot1.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1'],
      dtype='object')

In [107]:
dfTot2 = merge_dataframes_fillna(dfTot1, dfVTot, merge_on)


In [108]:
dfTot2.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'ID_VAL_1', 'V_VAL_1', 'V_VAL_2',
       'V_VAL_3', 'V_VAL_4', 'V_VAL_5', 'V_VAL_6', 'V_VAL_7', 'V_VAL_8',
       'V_VAL_9', 'V_VAL_10', 'V_VAL_11', 'V_VAL_12', 'V_VAL_13', 'V_VAL_14',
       'V_VAL_15', 'V_VAL_16', 'V_VAL_17', 'V_VAL_18', 'V_VAL_19', 'V_VAL_20',
       'V_VAL_21', 'V_VAL_22', 'V_VAL_23', 'V_VAL_24', 'V_VAL_25', 'V_VAL_26',
       'V_VAL_27', 'V_VAL_28', 'V_VAL_29', 'V_VAL_30', 'V_VAL_31', 'V_VAL_32'],
      dtype='object')

In [109]:
dfTot3 = dfTot2

In [110]:
dfTot3.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'ID_VAL_1', 'V_VAL_1', 'V_VAL_2',
       'V_VAL_3', 'V_VAL_4', 'V_VAL_5', 'V_VAL_6', 'V_VAL_7', 'V_VAL_8',
       'V_VAL_9', 'V_VAL_10', 'V_VAL_11', 'V_VAL_12', 'V_VAL_13', 'V_VAL_14',
       'V_VAL_15', 'V_VAL_16', 'V_VAL_17', 'V_VAL_18', 'V_VAL_19', 'V_VAL_20',
       'V_VAL_21', 'V_VAL_22', 'V_VAL_23', 'V_VAL_24', 'V_VAL_25', 'V_VAL_26',
       'V_VAL_27', 'V_VAL_28', 'V_VAL_29', 'V_VAL_30', 'V_VAL_31', 'V_VAL_32'],
      dtype='object')

In [111]:
#convert year variable in df2 to integer
dfMTot['ID_VAL']=dfMTot['ID_VAL'].astype(str)
dfTot3['ID_VAL']=dfTot3['ID_VAL'].astype(str)

In [112]:
dfTot4 = merge_dataframes_fillna(dfTot3, dfMTot, merge_on)


In [113]:
dfTot4

,ID_VAL,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,SS_VAL_1,SV_VAL_1,V_VAL_1,...,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72,ID_VAL_1
0,0,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
1,0,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
2,0,Arm,Arm,astar,int8_t,optO1-02,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
3,0,Leon3,Leon3,banker_algorithm,int16_t,optO1-00,32x32,NaN,NaN,1725.0,...,9123.916016,4056.985596,3445.193359,3842.214111,0.000000,10366.075195,7450.783691,0.0,5630.526855,0.0
4,0,Leon3,Leon3,banker_algorithm,int16_t,optO1-01,32x32,NaN,NaN,1725.0,...,9123.916016,4056.985596,3445.193359,3842.214111,0.000000,10366.075195,7450.783691,0.0,5630.526855,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-00,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1185,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-01,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1186,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-02,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1187,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,NaN,NaN,891.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0


In [114]:
dfTot4.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'V_VAL_1',
       ...
       'M_VAL_64', 'M_VAL_65', 'M_VAL_66', 'M_VAL_67', 'M_VAL_68', 'M_VAL_69',
       'M_VAL_70', 'M_VAL_71', 'M_VAL_72', 'ID_VAL_1'],
      dtype='object', length=114)

In [115]:
dfTot5 = dfTot4.fillna(0)

In [117]:
dfTot5.to_csv("TotalInput.csv", sep=';', index=False, encoding='utf-8')

In [45]:
####################### Useful Code ##########################################

In [46]:
# Matrix divided for Boards

In [47]:
df1 = dfTot5.loc[(dfTot5['BOARD'] == '5SGXEA7N2F45C1-Stratix-V')]
df1.to_csv("TotalParameterMatrixBambuValue_Stratix.csv", sep=';')

In [48]:
df2 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7a100t-1csg324-Artix-7')]
df2.to_csv("TotalParameterMatrixBambuValue_Artix.csv", sep=';')

In [49]:
df3 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7vx690t-3ffg1930-Virtex-7')]
df3.to_csv("TotalParameterMatrixBambuValue_Virtex.csv", sep=';')

In [50]:
df4 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7z020-1clg484-Zynq-7000')]
df4.to_csv("TotalParameterMatrixBambuValue_Zynq.csv", sep=';')

In [54]:
dfBamb = dfTot5.loc[(dfTot5['DEVICE'] == 'Bambu') & (dfTot5['BOARD'] == 'xc7a100t-1csg324-Artix-7')]

In [55]:
dfBamb.groupby(['DEVICE','BOARD','FUNCTION']).count()

DATA_TYPE  ID_VAL  DSP  BRAM  \
DEVICE BOARD                    FUNCTION                                      
Bambu  xc7a100t-1csg324-Artix-7 astar                100     100  100   100   
                                bellmanford          100     100  100   100   
                                bfsdfs               100     100  100   100   
                                binarysearch         100     100  100   100   
                                bubblesort           100     100  100   100   
                                dijkstra             100     100  100   100   
                                fft                  100     100  100   100   
                                fibcall              100     100  100   100   
                                floydwarshall        100     100  100   100   
                                gcd                  100     100  100   100   
                                insertionsort        100     100  100   100   
                                kruskal              100     100  100   100   
                                mergesort            100     100  100   100   
                                park_miller          100     100  100   100   
                                prime                100     100  100   100   
                                qsort                100     100  100   100   
                                quicksort            100     100  100   100   
                                selectionsort        100     100  100   100   
                                shell_sort           100     100  100   100   
                                sqrt                 100     100  100   100   
                                straight_sort        100     100  100   100   

                                               Flip_Flop  Area  min_Slack  \
DEVICE BOARD                    FUNCTION                                    
Bambu  xc7a100t-1csg324-Artix-7 astar                100   100        100   
                                bellmanford          100   100        100   
                                bfsdfs               100   100        100   
                                binarysearch         100   100        100   
                                bubblesort           100   100        100   
                                dijkstra             100   100        100   
                                fft                  100   100        100   
                                fibcall              100   100        100   
                                floydwarshall        100   100        100   
                                gcd                  100   100        100   
                                insertionsort        100   100        100   
                                kruskal              100   100        100   
                                mergesort            100   100        100   
                                park_miller          100   100        100   
                                prime                100   100        100   
                                qsort                100   100        100   
                                quicksort            100   100        100   
                                selectionsort        100   100        100   
                                shell_sort           100   100        100   
                                sqrt                 100   100        100   
                                straight_sort        100   100        100   

                                               Max_Freq  Time  Energy  ...  \
DEVICE BOARD                    FUNCTION                               ...   
Bambu  xc7a100t-1csg324-Artix-7 astar               100   100     100  ...   
                                bellmanford         100   100     100  ...   
                                bfsdfs              100   100     100  ...   
                                binarysearch        100   100     1

In [56]:
dfBamb.loc[dfTot5['FUNCTION'] == 'gcd'].groupby(['DEVICE','BOARD','FUNCTION','DATA_TYPE']).count()

,,,,ID_VAL,DSP,BRAM,Flip_Flop,Area,min_Slack,Max_Freq,Time,Energy,BugsDelivered,...,M_VAL_23,M_VAL_24,M_VAL_25,M_VAL_26,M_VAL_27,M_VAL_28,M_VAL_29,M_VAL_30,M_VAL_31,M_VAL_32
DEVICE,BOARD,FUNCTION,DATA_TYPE,,,,,,,,,,,,,,,,,,,,,
Bambu,xc7a100t-1csg324-Artix-7,gcd,int8_t,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
